**LOAD DATA FROM EXCEL FILE**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

# Load the data from an Excel file
data = pd.read_excel('initial_threshold_voltage_data.xlsx')

**PREPARE THE DATA**

In [ ]:
# Separate input features and target variable (Vti)
X = data[['Cycles', 'Vgp', 'dtp', 'Vge', 'dte', 'SONOSstate']]
y = data['Vti']

# Convert categorical feature 'SONOSstate' into numerical values (0 for erased, 1 for programmed)
X['SONOSstate'] = X['SONOSstate'].map({'erased': 0, 'programmed': 1})

# Split into training and testing sets
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

**PREPROCESS AND TRAIN THE MODEL**

In [ ]:
# Define the neural network model
model = Sequential()

# Input Layer
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Hidden Layer 1
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

# Hidden Layer 2
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

# Hidden Layer 3
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Output Layer
model.add(Dense(1, activation='linear'))  # Linear activation for regression

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

# Learning rate reduction callback
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=10, factor=0.5, min_lr=0.00001)

# Train the model
history = model.fit(X_train_scaled, y_train,
                    validation_data=(X_test_scaled, y_test),
                    epochs=200,
                    batch_size=32,
                    callbacks=[lr_reduction])

# Save the trained model
model.save('initial_threshold_voltage_model.h5')

**EVALUATE THE MODEL ON TESTING DATA**

In [ ]:
# Evaluate the model
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f'Mean Absolute Error on test set: {mae}')

# Predictions
y_pred = model.predict(X_test_scaled)

# Display sample predictions
print("Predicted vs Actual Initial Threshold Voltage Values:")
for i in range(5):
    print(f"Sample {i+1}: Predicted = {y_pred[i][0]:.4f}, Actual = {y_test.iloc[i]:.4f}")

**PLOT LOSS VS EPOCHS, ACTUAL VS PREDICTED VALUES, RESIDUALS**

In [ ]:
# Plot Loss vs Epochs
def plot_loss(history):
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss vs Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_loss(history)

# Plot Actual vs Predicted
plt.scatter(y_test, y_pred, color='blue', label='Predicted vs Actual')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', label='Perfect Prediction Line')
plt.title('Actual vs Predicted Initial Threshold Voltage')
plt.xlabel('Actual Initial Threshold Voltage')
plt.ylabel('Predicted Initial Threshold Voltage')
plt.legend()
plt.grid(True)
plt.show()

# Compute Residuals
residuals = y_test - y_pred.flatten()

# Plot Residuals
plt.scatter(y_test, residuals, color='purple', alpha=0.6)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Actual Initial Threshold Voltage')
plt.xlabel('Actual Initial Threshold Voltage')
plt.ylabel('Residuals')
plt.grid(True)
plt.show()

# Print Residual Statistics
print(f'Mean of Residuals: {np.mean(residuals):.4f}')
print(f'Standard Deviation of Residuals: {np.std(residuals):.4f}')